In [2]:
    import pandas as pd

    df = pd.read_csv('data/garage_data.csv')

    df.head()

,REFERENCE_NUM,PRODUCT_INDEX,VEHICLE_MODEL_CODE,CLAIMNO,CLAIM_NO,PARTNO,PARTDESCRIPTION,TOTAL_AMOUNT,NUM_CLAIM_NO
0,202001050026449,N000706,23103.0,'23002031940000000001,C230020319400,64520M56R00,"PANEL COMP, RR LAMP HOUSE, L",285.15,2.300203e+19
1,202001050026449,N000706,23103.0,'23002031940000000001,C230020319400,84681M56R00,"WEATHERSTRIP, TRUNK LID OPNG",484.37,2.300203e+19
2,202001050026449,N000706,23103.0,'23002031940000000001,C230020319400,65700M56R00,PANEL ASSY TRUNK LID,2492.18,2.300203e+19
3,202001050026449,N000706,23103.0,'23002031940000000001,C230020319400,35750M56R00,"LAMP, UNIT RR COMB LH",1915.25,2.300203e+19
4,202012250082721,N000706,30135.0,'23002031863800000001,C230020318638,31100M86J30,"MOTOR ASSY, STARTING",3613.28,2.300203e+19


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365086 entries, 0 to 365085
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   REFERENCE_NUM       365086 non-null  int64  
 1   PRODUCT_INDEX       365086 non-null  object 
 2   VEHICLE_MODEL_CODE  365086 non-null  float64
 3   CLAIMNO             365086 non-null  object 
 4   CLAIM_NO            365086 non-null  object 
 5   PARTNO              365086 non-null  object 
 6   PARTDESCRIPTION     365086 non-null  object 
 7   TOTAL_AMOUNT        365086 non-null  float64
 8   NUM_CLAIM_NO        365086 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 25.1+ MB


In [4]:
df.describe()

,REFERENCE_NUM,VEHICLE_MODEL_CODE,TOTAL_AMOUNT,NUM_CLAIM_NO
count,3.650860e+05,365086.000000,365086.000000,3.650860e+05
mean,2.020755e+14,29012.884044,1154.241787,2.300213e+19
std,8.998028e+10,8771.164294,2064.386175,8.424312e+13
min,2.018092e+14,11587.000000,0.390000,2.300201e+19
25%,2.020082e+14,20646.000000,76.560000,2.300204e+19
50%,2.021011e+14,30557.000000,414.060000,2.300211e+19
75%,2.021111e+14,36542.000000,1476.560000,2.300221e+19
max,2.024011e+14,47464.000000,155390.620000,2.300235e+19


In [5]:
# Convert scientific notation to integer
df['REFERENCE_NUM'] = df['REFERENCE_NUM'].astype('int64')
df['NUM_CLAIM_NO'] = df['NUM_CLAIM_NO'].astype('int64')

# Remove leading apostrophes in CLAIMNO
df['CLAIMNO'] = df['CLAIMNO'].astype(str).str.replace("'", "")

# Convert VEHICLE_MODEL_CODE to integer
df['VEHICLE_MODEL_CODE'] = df['VEHICLE_MODEL_CODE'].astype('int64')

# Drop duplicates
df.drop_duplicates(inplace=True)

# Handle missing values (Fill NaN with appropriate values or drop rows)
df.fillna("Unknown", inplace=True)
# Save cleaned data
df.to_csv("cleaned_data.csv", index=False)

print("Data cleaning complete. Cleaned file saved as garage_data_cleaned.csv")


Data cleaning complete. Cleaned file saved as cleaned_data.csv


In [20]:
!pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 17.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Ensure stopwords are available
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_special_characters(text):
    """
    Remove special characters, retaining only alphanumerics and whitespace.
    """
    return re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

def expand_abbreviations(text):
    """
    Expand common abbreviations.
    Example: 'assy' -> 'assembly', 'lh' -> 'left hand'
    """
    abbrev_dict = {
        'assy': 'assembly',
        'lh': 'left hand'
    }
    for abbr, full in abbrev_dict.items():
        text = re.sub(r'\b' + re.escape(abbr) + r'\b', full, text)
    return text

def standardize_positions(text):
    """
    Standardize positional terms using regex.
    Example: 'r' -> 'right', 'fr' -> 'front right'
    """
    pos_dict = {
        'r': 'right',
        'fr': 'front right'
    }
    for abbr, full in pos_dict.items():
        text = re.sub(r'\b' + re.escape(abbr) + r'\b', full, text)
    return text

def remove_stopwords(text):
    """
    Tokenize the text and remove stopwords.
    """
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return " ".join(filtered_tokens)

def clean_text(text):
    """
    Apply a series of text transformations:
    - Convert to lowercase
    - Remove special characters
    - Remove extra whitespace
    - Expand abbreviations
    - Standardize positional terms
    - Tokenize and remove stopwords
    """
    if pd.isnull(text):
        return text
    text = str(text).lower()  # Convert to lowercase
    text = remove_special_characters(text)
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    text = expand_abbreviations(text)
    text = standardize_positions(text)
    text = remove_stopwords(text)
    return text

# Read CSV file
df = pd.read_csv('data/garage_data.csv')

# Original cleaning steps
df['REFERENCE_NUM'] = df['REFERENCE_NUM'].astype('int64')
df['NUM_CLAIM_NO'] = df['NUM_CLAIM_NO'].astype('int64')
df['CLAIMNO'] = df['CLAIMNO'].astype(str).str.replace("'", "")
df['VEHICLE_MODEL_CODE'] = df['VEHICLE_MODEL_CODE'].astype('int64')
df.drop_duplicates(inplace=True)
df.fillna("Unknown", inplace=True)

# Apply additional text cleaning if the DESCRIPTION column exists
if 'PARTDESCRIPTION' in df.columns:
    df['PARTDESCRIPTION'] = df['PARTDESCRIPTION'].apply(clean_text)
else:
    print("Column 'DESCRIPTION' not found. Please adjust the column name as necessary.")

# Save cleaned data
df.to_csv("cleaned_data.csv", index=False)
print("Data cleaning complete. Cleaned file saved as cleaned_data.csv")


[nltk_data] Downloading package stopwords to C:\Users\Aman
[nltk_data]     Jaiswal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data cleaning complete. Cleaned file saved as cleaned_data.csv


In [6]:
sd = pd.read_csv('data/surveyor_data.csv', dtype=str)

sd.head()


,REFERENCE_NUM,PRODUCT_INDEX,VEHICLE_MODEL_CODE,CLAIMNO,NUM_CLAIM_NO,TXT_PARTS_GROUP_NAME,TXT_PARTS_NAME,TOTAL_AMOUNT,NUM_PART_CODE
0,202108260089768,N000706,23139.0,'23002200306800000001,2.300220e+19,Rubber Plastic Parts,Emblem|Logo Rear Middle,189.05,1086
1,202107090087260,N000706,38123.0,'23002149113800000001,2.300215e+19,Rubber Plastic Parts,Bumper Rear Assembly,1863.00,1007
2,202112040039636,N000706,17662.0,'23002150489000000001,2.300215e+19,Glass Parts,Windshield Glass Front,2860.00,1003
3,202112040039636,N000706,17662.0,'23002150489000000001,2.300215e+19,Rubber Plastic Parts,Moulding Front Windshield,296.00,1035
4,202112040039636,N000706,17662.0,'23002150489000000001,2.300215e+19,Metalic Parts,Bonnet|Hood Assembly,2628.00,1002


In [7]:
sd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945216 entries, 0 to 945215
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   REFERENCE_NUM         945216 non-null  int64  
 1   PRODUCT_INDEX         945216 non-null  object 
 2   VEHICLE_MODEL_CODE    945016 non-null  float64
 3   CLAIMNO               945216 non-null  object 
 4   NUM_CLAIM_NO          945216 non-null  float64
 5   TXT_PARTS_GROUP_NAME  945216 non-null  object 
 6   TXT_PARTS_NAME        945216 non-null  object 
 7   TOTAL_AMOUNT          945216 non-null  float64
 8   NUM_PART_CODE         945216 non-null  int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 64.9+ MB


In [18]:
cleaned_surveyor_data = pd.read_csv('surveyor_data_cleaned.csv')
cleaned_garage_data = pd.read_csv('garage_data_cleaned.csv')


cleaned_surveyor_data.head()
# cleaned_garage_data.head()

,REFERENCE_NUM,PRODUCT_INDEX,VEHICLE_MODEL_CODE,CLAIMNO,NUM_CLAIM_NO,TXT_PARTS_GROUP_NAME,TXT_PARTS_NAME,TOTAL_AMOUNT,NUM_PART_CODE
0,202108260089768,N000706,23139.0,23002200306800000001,2.300220e+19,Rubber Plastic Parts,Emblem|Logo Rear Middle,189.05,1086
1,202107090087260,N000706,38123.0,23002149113800000001,2.300215e+19,Rubber Plastic Parts,Bumper Rear Assembly,1863.00,1007
2,202112040039636,N000706,17662.0,23002150489000000001,2.300215e+19,Glass Parts,Windshield Glass Front,2860.00,1003
3,202112040039636,N000706,17662.0,23002150489000000001,2.300215e+19,Rubber Plastic Parts,Moulding Front Windshield,296.00,1035
4,202112040039636,N000706,17662.0,23002150489000000001,2.300215e+19,Metalic Parts,Bonnet|Hood Assembly,2628.00,1002


In [15]:
cleaned_surveyor_data.info()

print("<-------------------------------------------->")

cleaned_garage_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945208 entries, 0 to 945207
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   REFERENCE_NUM         945208 non-null  int64  
 1   PRODUCT_INDEX         945208 non-null  object 
 2   VEHICLE_MODEL_CODE    945208 non-null  float64
 3   CLAIMNO               945208 non-null  object 
 4   NUM_CLAIM_NO          945208 non-null  float64
 5   TXT_PARTS_GROUP_NAME  945208 non-null  object 
 6   TXT_PARTS_NAME        945208 non-null  object 
 7   TOTAL_AMOUNT          945208 non-null  float64
 8   NUM_PART_CODE         945208 non-null  int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 64.9+ MB
<-------------------------------------------->
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364567 entries, 0 to 364566
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   ----- 

In [28]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Ensure the NLTK stopwords are downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_special_characters(text):
    """
    Remove special characters, keeping only alphanumeric characters and whitespace.
    """
    return re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

def expand_abbreviations(text):
    """
    Expand common abbreviations in the text.
    Example: 'assy' -> 'assembly', 'lh' -> 'left hand'
    """
    abbrev_dict = {
        'assy': 'assembly',
        'lh': 'left hand'
    }
    for abbr, full in abbrev_dict.items():
        text = re.sub(r'\b' + re.escape(abbr) + r'\b', full, text)
    return text

def standardize_positions(text):
    """
    Standardize positional terms using regex.
    Example: 'r' -> 'right', 'fr' -> 'front right'
    """
    pos_dict = {
        'r': 'right',
        'fr': 'front right'
    }
    for abbr, full in pos_dict.items():
        text = re.sub(r'\b' + re.escape(abbr) + r'\b', full, text)
    return text

def remove_stopwords(text):
    """
    Tokenize the text and remove common stopwords.
    """
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return " ".join(filtered_tokens)

def clean_text(text):
    """
    Apply a series of text transformations:
    - Convert to lowercase
    - Remove special characters
    - Remove extra whitespace
    - Expand abbreviations
    - Standardize positional terms
    - Remove stopwords
    """
    if pd.isnull(text):
        return text
    text = text.lower()
    text = remove_special_characters(text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = expand_abbreviations(text)
    text = standardize_positions(text)
    text = remove_stopwords(text)
    return text

# Load surveyor data with all columns as strings
sd = pd.read_csv('data/surveyor_data.csv', dtype=str)

# Convert columns that might be in scientific notation to Int64
sci_notation_cols = ['REFERENCE_NUM', 'NUM_CLAIM_NO']
for col in sci_notation_cols:
    sd[col] = pd.to_numeric(sd[col], errors='coerce')  # Convert to numeric
    sd[col] = sd[col].fillna(0).astype('int64', errors='ignore')  # Fill NaN and cast safely
for col in sci_notation_cols:
    sd[col] = pd.to_numeric(sd[col], errors='coerce').fillna(0)
    if sd[col].dtype == 'float64':
        sd[col] = sd[col].astype('int64', errors='ignore')
# Ensure CLAIMNO values are properly formatted (remove leading apostrophes)
sd['CLAIMNO'] = sd['CLAIMNO'].str.replace("'", "", regex=True)

# Trim whitespace from all string columns
sd = sd.map(lambda x: x.strip() if isinstance(x, str) else x)

# Drop duplicate rows if any exist
sd = sd.drop_duplicates()

# Fill missing values for VEHICLE_MODEL_CODE and drop rows missing TXT_PARTS_NAME
sd['VEHICLE_MODEL_CODE'] = sd['VEHICLE_MODEL_CODE'].fillna(0)
sd = sd.dropna(subset=['TXT_PARTS_NAME'])

# Apply text cleaning to the TXT_PARTS_NAME column
sd['TXT_PARTS_NAME'] = sd['TXT_PARTS_NAME'].apply(clean_text)

# Save the cleaned dataset
sd.to_csv('surveyor_data_cleanedd.csv', index=False)

print("Data cleaning complete. Saved as 'surveyor_data_cleaned.csv'.")


[nltk_data] Downloading package stopwords to C:\Users\Aman
[nltk_data]     Jaiswal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data cleaning complete. Saved as 'surveyor_data_cleaned.csv'.
